In [13]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import MidTermFeatures
import os
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr

def extract_global_features(filepath):
    [Fs, x] = audioBasicIO.read_audio_file(filepath)
    x = audioBasicIO.stereo_to_mono(x)
    
    # Extraction des attributs à moyen terme
    mid_term_features, short_term_features, mid_feature_names = MidTermFeatures.mid_feature_extraction(x, Fs, 1.0 * Fs, 1.0 * Fs, 0.050 * Fs, 0.025 * Fs)
    
    # Calcul des attributs globaux (moyenne et écart-type des attributs à moyen terme)
    mean_features = np.mean(mid_term_features, axis=1)
    std_features = np.std(mid_term_features, axis=1)
    global_features = np.concatenate((mean_features, std_features))
    
    return global_features

In [17]:
def process_directory(directory_path):
    audio_files = [f for f in os.listdir(directory_path) if f.endswith('.mp3') or f.endswith('.wav')]
    all_global_features = []
    
    for audio_file in audio_files:
        filepath = os.path.join(directory_path, audio_file)
        global_features = extract_global_features(filepath)
        all_global_features.append(global_features)
    
    return np.array(all_global_features), audio_files


In [18]:
def calculate_distances(global_features, reference_index=0):
    distances = []
    for i in range(len(global_features)):
        if i != reference_index:
            dist = euclidean(global_features[reference_index], global_features[i])
            distances.append((i, dist))
    return sorted(distances, key=lambda x: x[1])

def rank_tracks(distances):
    return [x[0] for x in distances]

def evaluate_ranking(sorted_indices, ground_truth):
    rho, _ = spearmanr(sorted_indices, ground_truth)
    return rho

In [19]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import MidTermFeatures
import os
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr

# Paramètres d'entrée
directory_path = "data"
ground_truth = [1, 2, 3, 4, 5, 6, 7]

# Extraction des caractéristiques audio
global_features, audio_files = process_directory(directory_path)

# Calcul des distances L2 par rapport au morceau 0
distances = calculate_distances(global_features)

# Classement des morceaux par distance croissante
sorted_indices = rank_tracks(distances)

# Évaluation du classement
rho = evaluate_ranking(sorted_indices, ground_truth)

print("Classement des morceaux par distance croissante avec le morceau 0:")
for idx in sorted_indices:
    print(f"Morceau {idx} avec distance {dict(distances)[idx]}")

print("Corrélation de Spearman:", rho)
print("Classement de référence:", ground_truth)



Classement des morceaux par distance croissante avec le morceau 0:
Morceau 1 avec distance 2.287214932514865
Morceau 2 avec distance 2.461829299035586
Morceau 3 avec distance 4.199040157866551
Morceau 5 avec distance 4.275541192050813
Morceau 6 avec distance 6.547843959666771
Morceau 4 avec distance 6.6437562220340345
Morceau 7 avec distance 7.172492974405545
Corrélation de Spearman: 0.8928571428571429
Classement de référence: [1, 2, 3, 4, 5, 6, 7]
